In [1]:
import pandas as pd 
import numpy as np 
#np.random.bit_generator = np.random._bit_generator
import pyodbc
from datetime import datetime, timedelta
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy
import nltk
import timeit
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
# gensim
from gensim import corpora, models, similarities, matutils
import unicodedata

from sklearn.feature_extraction.text import TfidfVectorizer
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('punkt')
nltk.download("stopwords")

C:\Users\PCC2\.conda\envs\tenstest\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\PCC2\.conda\envs\tenstest\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\PCC2\.conda\envs\tenstest\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\PCC2\.conda\envs\tenstest\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


True

In [3]:
server = 't5.database.windows.net'
server='DESKTOP-ASD2RK7\SQLEXPRESS'
database = 'T5' 
username = 'T5' 
password = 'My404Data' 
database = 'Riyadh2'
username ='sab'
password = 'tata2015'
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
df= pd.read_sql("select  * from texts inner join articles on t_aid = a_id where a_words>150 and a_words <501 ",cnxn)
df.to_csv("texts3.csv",encoding='utf-8-sig')

In [ ]:
df2= pd.read_sql("select  * from Articles",cnxn)
df2.to_csv("articles.csv",encoding='utf-8-sig')
df2

In [ ]:
df = pd.read_csv("texts.csv")
df

In [ ]:
df3 = pd.merge(df2, df, how='inner', left_on='A_ID', right_on='T_AID')
df3

In [8]:
df3 = df

In [9]:
df3['Words_Number'] = df3['T_Text'].astype(str).apply(lambda x : len(x.split()))
df3['T_Text'] = df3['T_Text'].apply(lambda x:unicodedata.normalize("NFKD", str(x)).replace('\n', ''))
# Wrong Texts (i.e. very long/short texts)
df3 = df3[((df3['Words_Number'] > 150) & (df3['Words_Number'] < 500))]
df3 = df3.reset_index()
df3 = df3.drop_duplicates(subset ="T_Text")
#df3
Articles = list(df3['T_Text'].tolist())

In [11]:
wordtoken = word_tokenize(' '.join(Articles))

In [12]:
len(wordtoken)

37725320

In [14]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words = 'arabic',token_pattern='\\b[\u0621-\u064A][\u0621-\u064A]+\\b')
#count_vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words = stopwords.words('arabic'),token_pattern='\\b[\u0621-\u064A][\u0621-\u064A]+\\b')
 
X = count_vectorizer.fit_transform(Articles)
#cols = count_vectorizer.get_feature_names_out()
cols = count_vectorizer.get_feature_names()


C:\Users\PCC2\.conda\envs\tenstest\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [16]:

#print(cols[100064])
#X.toarray()
# without stop words removal we got 138,561
# using arabic stop words we got 138,108
#print(X)
#print(X[1:1])


In [17]:
X.shape

(137352, 10792609)

In [18]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [19]:
doc_word = count_vectorizer.transform(Articles).transpose()

In [21]:
ab  = pd.DataFrame(doc_word.toarray(), count_vectorizer.get_feature_names())
ab['Total'] = ab.sum(axis=1)
ab = ab.sort_values('Total',ascending=False)
ab.head(60)
#ab2
ab2.shape
#ab2.max()
#ab2.argmax()
#ab[ab2.argmax():ab2.argmax()+1]

MemoryError: Unable to allocate 10.8 TiB for an array with shape (10792609, 137352) and data type int64

In [ ]:
doc_word.shape

In [22]:
corpus = matutils.Sparse2Corpus(doc_word)
#models.LdaModel()

In [23]:
lda = models.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, passes=3,iterations=100)

2021-11-10 22:45:12,273 : INFO : using symmetric alpha at 0.1
2021-11-10 22:45:12,275 : INFO : using symmetric eta at 0.1
2021-11-10 22:45:13,599 : INFO : using serial LDA version on this node
2021-11-10 22:45:22,294 : INFO : running online (multi-pass) LDA training, 10 topics, 3 passes over the supplied corpus of 137352 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 100x with a convergence threshold of 0.001000
2021-11-10 22:45:22,776 : INFO : PROGRESS: pass 0, at document #2000/137352
2021-11-10 22:45:30,321 : INFO : merging changes from 2000 documents into a model of 137352 documents
2021-11-10 22:45:34,973 : INFO : topic #6 (0.100): 0.006*"الا" + 0.002*"لى" + 0.002*"كورونا" + 0.001*"با" + 0.001*"خلال" + 0.001*"المملكة" + 0.001*"والا" + 0.001*"فيروس" + 0.001*"وذلك" + 0.001*"ية"
2021-11-10 22:45:35,073 : INFO : topic #1 (0.100): 0.013*"الا" + 0.002*"لى" + 0.002*"كورونا" + 0.002*"خلال" + 0.002*"المملكة" + 0.001*"فيروس" + 0.0

2021-11-10 22:46:44,227 : INFO : merging changes from 2000 documents into a model of 137352 documents
2021-11-10 22:46:48,952 : INFO : topic #7 (0.100): 0.014*"الا" + 0.005*"زة" + 0.003*"سراي" + 0.003*"لى" + 0.002*"الاحتلال" + 0.002*"با" + 0.002*"الا سراي" + 0.002*"الفلسطينية" + 0.002*"الجاي زة" + 0.001*"الفلسطيني"
2021-11-10 22:46:49,070 : INFO : topic #1 (0.100): 0.001*"الا" + 0.000*"وريال" + 0.000*"ماراتي" + 0.000*"الا ماراتي" + 0.000*"وريال مدريد" + 0.000*"غينيا" + 0.000*"مليون مريكي" + 0.000*"ولفرهامبتون" + 0.000*"لى" + 0.000*"واندرارز"
2021-11-10 22:46:49,202 : INFO : topic #9 (0.100): 0.025*"الا" + 0.006*"الفريق" + 0.005*"بن" + 0.005*"لى" + 0.005*"الهلال" + 0.004*"مير" + 0.003*"ول" + 0.003*"محمد" + 0.003*"مام" + 0.003*"المباراة"
2021-11-10 22:46:49,326 : INFO : topic #0 (0.100): 0.010*"الا" + 0.007*"سهم" + 0.003*"كثر" + 0.002*"السعودية" + 0.002*"نقطة" + 0.002*"الا سهم" + 0.002*"لى" + 0.002*"رامكو" + 0.001*"شر" + 0.001*"الا كثر"
2021-11-10 22:46:49,434 : INFO : topic #2 (0.100): 

2021-11-10 22:48:10,909 : INFO : topic #2 (0.100): 0.000*"فضية" + 0.000*"برونزية" + 0.000*"بزمن ثانية" + 0.000*"سباق متر" + 0.000*"الصناع" + 0.000*"القوى لذوي" + 0.000*"السعودي لعاب" + 0.000*"كراسي" + 0.000*"كراسي متحركة" + 0.000*"النخلي"
2021-11-10 22:48:11,044 : INFO : topic #5 (0.100): 0.023*"الا" + 0.020*"بن" + 0.010*"مير" + 0.010*"الله" + 0.008*"عبدالعزيز" + 0.006*"بن عبدالعزيز" + 0.006*"الا مير" + 0.005*"الحرمين" + 0.005*"الشريفين" + 0.005*"يس"
2021-11-10 22:48:11,196 : INFO : topic #4 (0.100): 0.042*"الا" + 0.011*"لى" + 0.004*"با" + 0.004*"نه" + 0.003*"وقال" + 0.003*"خلال" + 0.003*"والا" + 0.003*"يس" + 0.002*"المتحدة" + 0.002*"للا"
2021-11-10 22:48:11,296 : INFO : topic #3 (0.100): 0.021*"الا" + 0.011*"لى" + 0.006*"المملكة" + 0.006*"خلال" + 0.004*"ية" + 0.003*"السعودية" + 0.003*"والا" + 0.003*"العامة" + 0.003*"العام" + 0.003*"العمل"
2021-11-10 22:48:11,391 : INFO : topic #8 (0.100): 0.001*"سكني" + 0.001*"قطعة" + 0.001*"قطعة رض" + 0.001*"رض" + 0.001*"برنامج سكني" + 0.001*"الضريبة

2021-11-10 22:49:19,863 : INFO : topic #5 (0.100): 0.023*"بن" + 0.023*"الا" + 0.012*"مير" + 0.009*"عبدالعزيز" + 0.007*"الله" + 0.007*"الا مير" + 0.007*"بن عبدالعزيز" + 0.006*"السمو" + 0.005*"يس" + 0.005*"محمد"
2021-11-10 22:49:19,958 : INFO : topic #6 (0.100): 0.000*"بكمية" + 0.000*"الترسية" + 0.000*"الشعير" + 0.000*"سسة خلال" + 0.000*"تجارة الحبوب" + 0.000*"استعراض سعار" + 0.000*"باستثناء كندا" + 0.000*"ترسية الدفعة" + 0.000*"المتعاقد عليها" + 0.000*"والشمالية باستثناء"
2021-11-10 22:49:20,049 : INFO : topic #2 (0.100): 0.000*"برونزية" + 0.000*"فضية" + 0.000*"النخلي" + 0.000*"سباق متر" + 0.000*"السعودي لعاب" + 0.000*"القوى لذوي" + 0.000*"عبدالرحمن القرشي" + 0.000*"علي النخلي" + 0.000*"الجنيدل" + 0.000*"ميداليات برونزية"
2021-11-10 22:49:20,131 : INFO : topic #0 (0.100): 0.007*"سهم" + 0.004*"الا" + 0.004*"كثر" + 0.003*"الا سهم" + 0.002*"نقطة" + 0.002*"السعودية" + 0.002*"شركات" + 0.002*"سهم شركات" + 0.002*"نماء" + 0.002*"الا كثر"
2021-11-10 22:49:20,233 : INFO : topic #8 (0.100): 0.001*

2021-11-10 22:50:31,096 : INFO : topic diff=0.003145, rho=0.204124
2021-11-10 22:50:31,306 : INFO : PROGRESS: pass 0, at document #50000/137352
2021-11-10 22:50:37,018 : INFO : merging changes from 2000 documents into a model of 137352 documents
2021-11-10 22:50:41,743 : INFO : topic #7 (0.100): 0.008*"زة" + 0.004*"سراي" + 0.002*"الجاي زة" + 0.002*"الجاي" + 0.002*"الاحتلال" + 0.002*"الا سراي" + 0.002*"جاي زة" + 0.002*"جاي" + 0.002*"يلي" + 0.002*"الفلسطيني"
2021-11-10 22:50:41,892 : INFO : topic #3 (0.100): 0.021*"الا" + 0.010*"لى" + 0.006*"المملكة" + 0.005*"خلال" + 0.004*"ية" + 0.004*"السعودية" + 0.003*"العامة" + 0.003*"العمل" + 0.003*"العام" + 0.003*"والا"
2021-11-10 22:50:41,995 : INFO : topic #1 (0.100): 0.000*"مليون مريكي" + 0.000*"وريال" + 0.000*"وريال مدريد" + 0.000*"ولفرهامبتون" + 0.000*"واندرارز" + 0.000*"ولفرهامبتون واندرارز" + 0.000*"غينيا" + 0.000*"رودجرز" + 0.000*"الا" + 0.000*"بريندان"
2021-11-10 22:50:42,099 : INFO : topic #9 (0.100): 0.026*"الا" + 0.006*"الفريق" + 0.004*

...

2021-11-10 23:23:59,091 : INFO : topic diff=0.003159, rho=0.118117
2021-11-10 23:23:59,294 : INFO : PROGRESS: pass 2, at document #92000/137352
2021-11-10 23:24:04,801 : INFO : merging changes from 2000 documents into a model of 137352 documents
2021-11-10 23:24:09,449 : INFO : topic #0 (0.100): 0.002*"سهم" + 0.002*"التهاني" + 0.001*"الا نماء" + 0.001*"الاكتتاب" + 0.001*"نماء" + 0.001*"مارة مس" + 0.001*"الا سهم" + 0.001*"مسية" + 0.001*"لفرع" + 0.001*"كثر"
2021-11-10 23:24:09,541 : INFO : topic #8 (0.100): 0.002*"ضريبة" + 0.002*"ضريبة القيمة" + 0.002*"الضريبة" + 0.001*"الاستقدام" + 0.001*"الفرص الوظيفية" + 0.001*"التنظيمات" + 0.001*"للضريبة" + 0.001*"سعودي الجنسية" + 0.001*"الصلاحية" + 0.001*"بارتكاب"
2021-11-10 23:24:09,678 : INFO : topic #5 (0.100): 0.023*"بن" + 0.020*"الا" + 0.012*"مير" + 0.011*"الله" + 0.010*"عبدالعزيز" + 0.007*"بن عبدالعزيز" + 0.007*"الا مير" + 0.006*"الحرمين" + 0.006*"الشريفين" + 0.006*"الحرمين الشريفين"
2021-11-10 23:24:09,794 : INFO : topic #3 (0.100): 0.021*"ال

2021-11-10 23:25:17,946 : INFO : topic #1 (0.100): 0.000*"مليون مريكي" + 0.000*"وريال" + 0.000*"وريال مدريد" + 0.000*"مريكي مقابل" + 0.000*"مريكي خلال" + 0.000*"مريكي للمساهمة" + 0.000*"الصادرات لى" + 0.000*"منحة" + 0.000*"منحة بمبلغ" + 0.000*"وتمويل بمبلغ"
2021-11-10 23:25:18,235 : INFO : topic diff=0.002532, rho=0.118117
2021-11-10 23:25:18,438 : INFO : PROGRESS: pass 2, at document #104000/137352
2021-11-10 23:25:23,965 : INFO : merging changes from 2000 documents into a model of 137352 documents
2021-11-10 23:25:28,561 : INFO : topic #6 (0.100): 0.000*"بكمية" + 0.000*"ترسية الدفعة" + 0.000*"جراءات ترسية" + 0.000*"متخصصة تجارة" + 0.000*"تجارة الحبوب" + 0.000*"تمت المو" + 0.000*"دعوة شركة" + 0.000*"المتعاقد عليها" + 0.000*"للمهتمين استعراض" + 0.000*"لمناقصات"
2021-11-10 23:25:28,689 : INFO : topic #7 (0.100): 0.003*"تنظيم داعش" + 0.003*"سراي" + 0.003*"الا سراي" + 0.002*"الاحتلال" + 0.002*"يلية" + 0.001*"سراي يلي" + 0.001*"سراي يلية" + 0.001*"التمنيات" + 0.001*"مقاتلات" + 0.001*"يلي"


2021-11-10 23:26:25,592 : INFO : topic #8 (0.100): 0.001*"الضريبة" + 0.001*"التنظيمات" + 0.001*"الاستقدام" + 0.001*"سعودي الجنسية" + 0.001*"ضريبة" + 0.001*"الفرص الوظيفية" + 0.001*"ضريبة القيمة" + 0.001*"الصلاحية" + 0.000*"بارتكاب" + 0.000*"حجار"
2021-11-10 23:26:25,878 : INFO : topic diff=0.002523, rho=0.118117
2021-11-10 23:26:26,086 : INFO : PROGRESS: pass 2, at document #116000/137352
2021-11-10 23:26:31,566 : INFO : merging changes from 2000 documents into a model of 137352 documents
2021-11-10 23:26:36,167 : INFO : topic #2 (0.100): 0.000*"برونزية" + 0.000*"فضية" + 0.000*"بزمن ثانية" + 0.000*"سباق متر" + 0.000*"لمربط" + 0.000*"المهر" + 0.000*"فضية البطولة" + 0.000*"العاي لمربط" + 0.000*"برونزية البطولة" + 0.000*"وفي بطولة"
2021-11-10 23:26:36,292 : INFO : topic #3 (0.100): 0.021*"الا" + 0.009*"لى" + 0.006*"خلال" + 0.005*"المملكة" + 0.004*"ية" + 0.004*"العمل" + 0.003*"دارة" + 0.003*"والا" + 0.003*"العامة" + 0.003*"العام"
2021-11-10 23:26:36,443 : INFO : topic #4 (0.100): 0.040*"ال

2021-11-10 23:27:44,193 : INFO : topic #9 (0.100): 0.021*"الا" + 0.005*"الفريق" + 0.005*"الاتحاد" + 0.004*"ول" + 0.003*"السعودي" + 0.003*"النادي" + 0.003*"ندية" + 0.003*"القدم" + 0.003*"لى" + 0.003*"دارة"
2021-11-10 23:27:44,306 : INFO : topic #0 (0.100): 0.003*"التهاني" + 0.001*"مورينيو" + 0.001*"سهم" + 0.001*"مسية" + 0.001*"الا نماء" + 0.001*"مارة مس" + 0.001*"المساهمين" + 0.001*"الدولار" + 0.001*"المحرج" + 0.001*"التدريب التقني"
2021-11-10 23:27:44,592 : INFO : topic diff=0.002642, rho=0.118117
2021-11-10 23:27:44,801 : INFO : PROGRESS: pass 2, at document #128000/137352
2021-11-10 23:27:50,316 : INFO : merging changes from 2000 documents into a model of 137352 documents
2021-11-10 23:27:54,934 : INFO : topic #8 (0.100): 0.001*"الاستقدام" + 0.001*"سعودي الجنسية" + 0.001*"التنظيمات" + 0.001*"الفرص الوظيفية" + 0.001*"الضريبة" + 0.001*"بارتكاب" + 0.001*"الصلاحية" + 0.000*"ضريبة" + 0.000*"ويتميز" + 0.000*"حجار"
2021-11-10 23:27:55,033 : INFO : topic #5 (0.100): 0.024*"بن" + 0.020*"الا" 

2021-11-10 23:29:01,097 : INFO : topic #5 (0.100): 0.025*"بن" + 0.020*"الا" + 0.011*"مير" + 0.010*"الله" + 0.010*"عبدالعزيز" + 0.008*"الا مير" + 0.007*"بن عبدالعزيز" + 0.006*"يس" + 0.006*"محمد" + 0.006*"السمو"
2021-11-10 23:29:01,260 : INFO : topic #9 (0.100): 0.021*"الا" + 0.006*"الفريق" + 0.005*"الاتحاد" + 0.004*"النادي" + 0.004*"ول" + 0.003*"دارة" + 0.003*"القدم" + 0.003*"السعودي" + 0.003*"الموسم" + 0.003*"لى"
2021-11-10 23:29:01,544 : INFO : topic diff=0.001655, rho=0.118117
2021-11-10 23:29:01,623 : INFO : LdaModel lifecycle event {'msg': 'trained LdaModel(num_terms=10792609, num_topics=10, decay=0.5, chunksize=2000) in 2619.33s', 'datetime': '2021-11-10T23:29:01.622868', 'gensim': '4.1.2', 'python': '3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [ ]:
corpus = matutils.Sparse2Corpus(doc_word)
corpus

In [24]:
lda.print_topics()


2021-11-10 23:34:20,780 : INFO : topic #0 (0.100): 0.003*"التهاني" + 0.001*"الا نماء" + 0.001*"مسية" + 0.001*"المساهمين" + 0.001*"مارة مس" + 0.001*"مورينيو" + 0.001*"الدولار" + 0.001*"سهم" + 0.001*"الا مسية" + 0.001*"مصرف الا"
2021-11-10 23:34:20,862 : INFO : topic #1 (0.100): 0.000*"وريال" + 0.000*"وريال مدريد" + 0.000*"مليون مريكي" + 0.000*"برشلونة وريال" + 0.000*"مريكي للمساهمة" + 0.000*"تقديم تمويل" + 0.000*"الاتحاد الاسباني" + 0.000*"اديداس" + 0.000*"تمويل بمبلغ" + 0.000*"واتلتيكو"
2021-11-10 23:34:20,942 : INFO : topic #2 (0.100): 0.000*"برونزية" + 0.000*"فضية" + 0.000*"بزمن ثانية" + 0.000*"سباق متر" + 0.000*"لوزن" + 0.000*"لوزن كجم" + 0.000*"فضية وزن" + 0.000*"برونزية وزن" + 0.000*"الحفرة" + 0.000*"كلغ"
2021-11-10 23:34:21,071 : INFO : topic #3 (0.100): 0.021*"الا" + 0.009*"لى" + 0.006*"خلال" + 0.005*"المملكة" + 0.004*"ية" + 0.004*"العمل" + 0.003*"العام" + 0.003*"دارة" + 0.003*"والا" + 0.003*"العامة"
2021-11-10 23:34:21,243 : INFO : topic #4 (0.100): 0.039*"الا" + 0.011*"لى" + 0

[(0,
  '0.003*"التهاني" + 0.001*"الا نماء" + 0.001*"مسية" + 0.001*"المساهمين" + 0.001*"مارة مس" + 0.001*"مورينيو" + 0.001*"الدولار" + 0.001*"سهم" + 0.001*"الا مسية" + 0.001*"مصرف الا"'),
 (1,
  '0.000*"وريال" + 0.000*"وريال مدريد" + 0.000*"مليون مريكي" + 0.000*"برشلونة وريال" + 0.000*"مريكي للمساهمة" + 0.000*"تقديم تمويل" + 0.000*"الاتحاد الاسباني" + 0.000*"اديداس" + 0.000*"تمويل بمبلغ" + 0.000*"واتلتيكو"'),
 (2,
  '0.000*"برونزية" + 0.000*"فضية" + 0.000*"بزمن ثانية" + 0.000*"سباق متر" + 0.000*"لوزن" + 0.000*"لوزن كجم" + 0.000*"فضية وزن" + 0.000*"برونزية وزن" + 0.000*"الحفرة" + 0.000*"كلغ"'),
 (3,
  '0.021*"الا" + 0.009*"لى" + 0.006*"خلال" + 0.005*"المملكة" + 0.004*"ية" + 0.004*"العمل" + 0.003*"العام" + 0.003*"دارة" + 0.003*"والا" + 0.003*"العامة"'),
 (4,
  '0.039*"الا" + 0.011*"لى" + 0.005*"با" + 0.004*"نه" + 0.003*"وقال" + 0.003*"والا" + 0.002*"يس" + 0.002*"ان" + 0.002*"خلال" + 0.002*"للا"'),
 (5,
  '0.025*"بن" + 0.020*"الا" + 0.011*"مير" + 0.010*"الله" + 0.010*"عبدالعزيز" + 0.008*"ا

In [25]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
#del lda_corpus

In [26]:
df[df['T_Text'].str.contains("ية ")].iloc[4].T_Text

'أودع صندوق تنمية الموارد البشرية (هدف)، اليوم (الأحد)، 445.9 مليون ريال، في الحسابات البنكية لأكثر من 294 ألف مستفيد ومستفيدة من المسجلين في برنامج إعانة البحث عن عمل وإعانة صعوبة الحصول على عمل، وذلك عن شهر مارس 2020م.وكشف "هدف"، أن عدد المستفيدين من إعانة البحث عن عمل بلغ 156,449 مستفيد ومستفيدة، بينما بلغ أعداد المستفيدين من برنامج إعانة صعوبة الحصول على عمل 137,893 مستفيد ومستفيدة، مبينا أن عدد من التحقوا بسوق العمل بالقطاع الخاص من مستفيدي برنامجي إعانة البحث عن عمل، وإعانة صعوبة الحصول على عمل لشهر مارس، بلغ 6,770 مستفيد ومستفيدة منهم 4,136 إناث و2,634 ذكور.ويقدم صندوق تنمية الموارد البشرية، برامج دعم تأهيل وتدريب، لمستفيدي ومستفيدات برنامجي إعانة البحث عن عمل، وإعانة صعوبة الحصول على عمل، بهدف تطوير مهاراتهم وقدراتهم، انطلاقاً من أهداف ورسالة الصندوق في تمكين أبناء وبنات الوطن من الالتحاق بسوق العمل والمشاركة في مسارات التنمية الوطنية الراهنة.ودعماً لتأهيل وتدريب القوى الوطنية من الجنسين، حرص "هدف" على إطلاق برامج محفزة للتوطين وكذلك عقد لقاءات وظيفية بالتع

In [27]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

In [29]:
df6 = pd.DataFrame(columns=['ArticleID','TopicID','Perc'],dtype='object')
def func(x):
    #pass
    #print(x['index'])
    global df6
    for x1 in x['D']:
        #print(x1[1])
        #df6.iloc[-1] = [0,x1[0],x1[1]]
        df6 = df6.append(pd.DataFrame(columns=['ArticleID','TopicID','Perc'],data=[[x['index'],x1[0],x1[1]]]))
        #df6.index = df6.index+1

In [30]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs[0:50]
df5 = pd.DataFrame(columns = ["D"],data=zip(lda_docs))
#a = np.array(lda_docs[2])
#a[:,1].max()

#df5['Perc'] = df5['D'].apply(lambda x:list(zip(x))[0][0][1])
df5 = df5.reset_index()
df5.apply(lambda x:func(x),axis=1)
df5.head()
df6.head()

ImportError: cannot import name 'is_url' from 'pandas.io.common' (C:\Users\PCC2\.conda\envs\tenstest\lib\site-packages\pandas\io\common.py)

  ArticleID TopicID      Perc
0         0       3  0.149825
0         0       4  0.113769
0         0       5  0.733569
0         1       3  0.290020
0         1       4  0.636602

In [34]:
df3

ImportError: cannot import name 'is_url' from 'pandas.io.common' (C:\Users\PCC2\.conda\envs\tenstest\lib\site-packages\pandas\io\common.py)

         index    T_ID   T_AID  \
0            0      58     104   
1            1      64     116   
2            2       7       2   
3            3       8       4   
4            4       9       6   
...        ...     ...     ...   
138247  140964  283758  567507   
138248  140965  283760  567511   
138249  140966  283765  567521   
138250  140967  283767  567525   
138251  140968  283768  567527   

                                                   T_Text    A_ID  \
0       قال صاحب السمو الملكي الأمير د. فيصل بن مشعل ...     104   
1       كشفت جولة نفذتها «الرياض» على مناطق عدة في محا...     116   
2       نشرت وكالة رويترز، اليوم الأحد الموافق 5 إبر...       2   
3       بدأت إدارة التعليم في محافظة وادي الدواسر ، ...       4   
4       أودع صندوق تنمية الموارد البشرية (هدف)، اليوم...       6   
...                                                   ...     ...   
138247  ترأس صاحب السمو الملكي الأمير فيصل بن سلمان ...  567507   
138248  عقد خادم الحرمين الشريفين الملك

In [42]:
#df6[df6['TopicID']==12]
df7 = pd.merge(df6,df3,how='left',left_on='ArticleID',right_on='index')
df7[df7['TopicID']==9][['ArticleID','TopicID','Perc','A_Cat','A_Title1','A_Title2','T_Text']]

#df7[df7['Index']==4][['ArticleID','TopicID','Perc','A_Cat','A_Title1','A_Title2','T_Text']]
#df7.head()
#df7.to_csv("articles_Topics1.csv",encoding='utf-8-sig')
df7.shape

(455250, 19)

In [ ]:
Articles[100]

In [49]:
x=lda.show_topics(num_topics=10, num_words=10,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Topics and Words
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

#Below Code Prints Only Words 
for topic,words in topics_words:
    print(" ".join(words))

0::['التهاني', 'الا نماء', 'مسية', 'المساهمين', 'مارة مس', 'مورينيو', 'الدولار', 'سهم', 'الا مسية', 'مصرف الا']
1::['وريال', 'وريال مدريد', 'مليون مريكي', 'برشلونة وريال', 'مريكي للمساهمة', 'تقديم تمويل', 'الاتحاد الاسباني', 'اديداس', 'تمويل بمبلغ', 'واتلتيكو']
2::['برونزية', 'فضية', 'بزمن ثانية', 'سباق متر', 'لوزن', 'لوزن كجم', 'فضية وزن', 'برونزية وزن', 'الحفرة', 'كلغ']
3::['الا', 'لى', 'خلال', 'المملكة', 'ية', 'العمل', 'العام', 'دارة', 'والا', 'العامة']
4::['الا', 'لى', 'با', 'نه', 'وقال', 'والا', 'يس', 'ان', 'خلال', 'للا']
5::['بن', 'الا', 'مير', 'الله', 'عبدالعزيز', 'الا مير', 'بن عبدالعزيز', 'يس', 'محمد', 'السمو']
6::['بكمية', 'ترسية الدفعة', 'متخصصة تجارة', 'تجارة الحبوب', 'جراءات ترسية', 'تمت المو', 'للمهتمين استعراض', 'للحبوب جراءات', 'لمناقصات', 'سعار الترسية']
7::['تنظيم داعش', 'الاحتلال', 'سراي', 'الا سراي', 'التمنيات', 'يلية', 'لبلاده', 'سراي يلية', 'البنك السعودي', 'سراي يلي']
8::['الاستقدام', 'سعودي الجنسية', 'التنظيمات', 'الفرص الوظيفية', 'بارتكاب', 'الصلاحية', 'الضريبة